# Building

If you have already built the executable as `./build/sdf-weighted-delaunay` feel free to skip this part.

Otherwise, set `build=True` to build. 

In [ ]:
build   = False
binpath = "./build/sdf-weighted-delaunay"

import os
if build:
    if not os.path.exists("./build"):
        os.mkdir("build")
    !cmake -B build -DCGAL_DIR=./cgal 
    !cmake --build build/ --parallel
    
if os.path.exists(binpath): 
    print("Executable Available")
else:
    print("Error: Executable not available")

# SDF sampling
The re-sampling requires a sdf function that can be evaluated. This example uses a ground truth mesh and the `signed_distance` function provided by the `gpytoolbox` package.

In [ ]:
import os
# Download the Armadillo from github if it is not available already
mesh_path = "armadillo.obj"
if not os.path.exists(mesh_path) and mesh_path == "armadillo.obj":
    import wget
    url = 'https://raw.githubusercontent.com/alecjacobson/common-3d-test-models/refs/heads/master/data/armadillo.obj'
    filename = wget.download(url)
    print()
    print(filename)
else:
    print("Found .obj file")

# Contouring

In [ ]:
N = 10
max_refinement = 10000

!{binpath} {mesh_path} {N} {max_refinement}